In [ ]:
#Automatic COVID detection from patient's CT scan folder
# In this code the CT selection algorithm has not been used

In [ ]:
# idea from https://github.com/mr7495/COVID-CT-Code
# developer email : mr7495@yahoo.com

In [ ]:
!nvidia-smi #show the allocated GPU

Sat Jul 10 22:35:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Connect your Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#upgrade and install essential libraries
#Install essential libraries
!pip install git+https://github.com/mr7495/RetinaNet

  Cloning https://github.com/mr7495/RetinaNet to /tmp/pip-req-build-k2ekmcky
  Running command git clone -q https://github.com/mr7495/RetinaNet /tmp/pip-req-build-k2ekmcky
  Running command git submodule update --init --recursive -q
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp37-cp37m-linux_x86_64.whl size=181952 sha256=e9115008f7cda6337c40ff041fbf22fcf3d86c86a52adbfeab0d05996f146d29
  Stored in directory: /tmp/pip-ephem-wheel-cache-xoi0bixc/wheels/08/6d/f5/38473102b36a5975e02e8f339fbf85bc4d1b1c7c80dc68a595
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13345 sha256=6e192ed889051c2398d945e852d7004cca9dff5d0bcc36353039936c56c20630
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet


In [ ]:
import keras
import numpy as np
import cv2
import os
import random
import shutil
import pandas as pd
import csv
import zipfile
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import RandomNormal
import keras.backend as k
from sklearn.utils import shuffle
import io
from PIL import Image as pil_image
from keras_retinanet import layers
import keras.backend as k
import keras_retinanet

In [ ]:
shape=(512,512,1) #shape of the dataset images (in TIFF format)

In [ ]:
!unzip /content/drive/MyDrive/COVID-CTset.zip -d /content/data

Archive:  /content/drive/MyDrive/COVID-CTset.zip
replace /content/data/COVID-CTset/covid1/patient1/SR_2/IM00001.tif? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#The fully automated patient classification network
Path='/content/drive/MyDrive/trained_networks/FPN-fold1.hdf5' # You can use this trained model : 'drive/My Drive/COVID-CTset/trained_networks/FPN-fold1.hdf5'
data_path= '/content/data/COVID-CTset/covid1/patient1/SR_2/'  #path to the folder containing patient's CT scan images

selected_imgs_path=[]
for r,d,f in os.walk(data_path): 
  for file in f:
    if '.tif' in file:
      selected_imgs_path.append(os.path.join(r,file)) #get the images path from the data_folder

k.clear_session() #clear keras backend
custom_object={'UpsampleLike': keras_retinanet.layers._misc.UpsampleLike} #Upsampling layer that was used in feature pyramid network
net=keras.models.load_model(Path, custom_objects=custom_object)


covid_label=0 # index of COVID-19 class 
normal_label=1  # index of normal class 
normal_predicted=0
covid_predicted=0
for img_name in selected_imgs_path: #read images
  img=cv2.imread(img_name,cv2.IMREAD_UNCHANGED)
  pred_ind=np.argmax(net.predict(np.expand_dims(np.expand_dims(img,axis=0),axis=3))[0]) #predicted label
  if pred_ind==normal_label:
    normal_predicted+=1
  else:
    covid_predicted+=1

all_imgs_num=len(selected_imgs_path) 
if covid_predicted>=0.1*all_imgs_num: #if at least 10% of a patient CT scans be classified as covid
  print('The patient is identified as infected to COVID-19')
else:
  print('The patient is identified as normal')

The patient is identified as infected to COVID-19


In [ ]:
print(all_imgs_num)
print(covid_predicted)

35
19
